### Influencer profile statistics 

Code for getting aggregated influencer statistics - can be run!

In [1]:
import pandas as pd
import numpy as np
import re

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import dash
import random
import pickle
from collections import defaultdict, Counter
import networkx as nx

from sklearn.cluster import DBSCAN, KMeans

In [2]:
influencers_df = pd.read_csv('../data/influencer_db_17112022.csv')
influencer_posts_df = pd.read_csv('../data/influencer_post_db_temp.csv')

In [3]:
influencer_posts_df['edge_media_to_caption'] = influencer_posts_df['edge_media_to_caption'].replace(np.nan, "")
influencer_posts_df

,username,post_type,display_url,is_video,edge_media_to_caption,edge_media_to_comment,taken_at_timestamp,edge_liked_by,edge_media_to_tagged_users,video_view_count
0,huinileee,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,"It’s day one, or one day. 🗓 \n\nYou’re always ...",31,1640772046,3052,['huawei.sg'],31901.0
1,huinileee,GraphSidecar,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,False,"It’s a privilege to be alive, smell flowers, h...",16,1668074169,2868,['itsyourlifesg'],0.0
2,huinileee,GraphSidecar,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,False,Spinning my way into all the upcoming deals! 🚲...,31,1667988125,5670,"['zalora', 'adidassg', 'synccycle']",0.0
3,huinileee,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,Not letting my mid-day bummer hit me! 🕕\n\nIt’...,18,1667707204,2190,"['fever.sg', 'ricola_sg']",15445.0
4,huinileee,GraphSidecar,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,False,Seas-ing the day today with a morning row! 🚣‍♀...,16,1667622415,4463,['pa.passionwave'],0.0
...,...,...,...,...,...,...,...,...,...,...
2176,shanepowxp,GraphSidecar,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,False,,16,1663919656,1407,"['newbalance', 'newbalancelifestyle']",2273.0
2177,shanepowxp,GraphImage,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,False,,25,1663491214,1968,[],2273.0
2178,shanepowxp,GraphImage,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,False,,28,1662091995,1430,"['kanghyungsuk_', 'honghuifang']",2273.0
2179,shanepowxp,GraphImage,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,False,,7,1662029814,1152,[],2273.0


In [4]:
import pickle
with open('data/profile_category_dict.pkl', 'rb') as f:
    company_cat = pickle.load(f)
company_cat

{'eric4708': 'Artist',
 'beyondtheisland': 'Entrepreneur',
 'brassliondistillery': 'Wine/spirits',
 'underarmoursg': 'Clothing (Brand)',
 'jbl_sg': 'Electronics',
 'the.smiling.afro': 'Swimming Pool & Hot Tub Service',
 'vannysaywhat': 'Product/service',
 'edmundchen.sg': 'Public figure',
 'banyantreehotels': 'Hotel',
 'kodakstyle_kr': 'Clothing (Brand)',
 'foryou.sg': 'Beauty, cosmetic & personal care',
 'buffbaby88': 'Personal blog',
 'pumasportstyle': 'Clothing (Brand)',
 'closetsingapore': 'Boutique Store',
 'revolve': 'Retail company',
 'luckykhoonsg': 'Home Improvement',
 'absoluteprsg': 'Product/service',
 'gemme.clothing': 'Clothing (Brand)',
 'tigerbalmsg': 'Pharmaceuticals',
 'chamberlaincoffee': 'Coffee shop',
 'jacob.mahler': 'Athlete',
 'ceramiqueaesthetics': 'Local business',
 'periperasingapore': 'Beauty, cosmetic & personal care',
 'aolsg': 'Community Organization',
 'dermalab_sg': 'Health/beauty',
 'v_shantipereira': 'Athlete',
 'sleepiingalien': 'Fictional Character',

In [23]:
current_influencer_posts = influencer_posts_df[influencer_posts_df['username'] == 'aglimpseofrach']
current_influencer_posts

,username,post_type,display_url,is_video,edge_media_to_caption,edge_media_to_comment,taken_at_timestamp,edge_liked_by,edge_media_to_tagged_users,video_view_count
183,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,Did you catch our @313somerset LIVE stream wit...,2,1658408460,185,['313somerset'],6895.0
184,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,TOOK UP DAT 3 MIN MAKEUP CHALLENGE😜😜😜 how did ...,3,1655385612,146,"['fentybeauty', 'charlottetilbury', 'lauramerc...",3610.0
185,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,A little eyeshadow video!!! Turn up the volume...,8,1650367221,185,"['hudabeauty', 'sephorasg', 'theprpeople']",3451.0
186,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,I’ve got a competitor in my house 😩😩😩 The +1 h...,17,1647602185,362,"['osim_sg', 'isaac.com.sg']",9043.0
187,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,TALK SOME MONEY 💰💸 we’re answering some money ...,2,1645708980,169,"['maybanksg', 'isaac.com.sg']",7459.0
188,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,<FENTY BEAUTY GIVEAWAY💓> not my usual style bu...,199,1645452074,291,['fentybeauty'],4887.0
189,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,"HAPPY XIN NIAN 😛🧧 as requested, here’s more GR...",3,1643699712,311,"['maybelline', 'charlottetilbury', 'lauramerci...",6662.0
190,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,"YOUTUBE: HOW WELL DO WE KNOW EACH OTHER, SPICY...",4,1641823000,390,"['isaac.com.sg', 'yomo_sg']",9433.0
191,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,"GRWM WINTER EDITION❄️ NO FOUNDATION, super hyd...",7,1641653169,178,"['urbandecaycosmetics', 'lauramercier', 'laroc...",5954.0
192,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,"❄️unwind with me, winter edition❄️\nThese prod...",1,1641118333,262,"['lorealparis', 'larocheposay', 'drunkelephant...",7339.0


In [27]:
def parse_caption(row):
    mentions = [re.sub('[^0-9a-zA-Z_\.]+', '', i[1:]) for i in row.edge_media_to_caption.split() if i.startswith('@')]
    hashtags = [i[1:] for i in row.edge_media_to_caption.split() if i.startswith('#')]
    all_mentions = set(mentions + eval(row['edge_media_to_tagged_users']))
    all_mentions = list(all_mentions)
    row['mentions'] = all_mentions
    row['hashtags'] = hashtags
    return row
    
current_influencer_posts = current_influencer_posts.apply(parse_caption, axis=1)
current_influencer_posts

,username,post_type,display_url,is_video,edge_media_to_caption,edge_media_to_comment,taken_at_timestamp,edge_liked_by,edge_media_to_tagged_users,video_view_count,mentions,hashtags
183,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,Did you catch our @313somerset LIVE stream wit...,2,1658408460,185,['313somerset'],6895.0,"[313somersets, 313somerset, theroycelee]",[]
184,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,TOOK UP DAT 3 MIN MAKEUP CHALLENGE😜😜😜 how did ...,3,1655385612,146,"['fentybeauty', 'charlottetilbury', 'lauramerc...",3610.0,"[heroinemakesg, charlottetilbury, clarinssg, b...",[]
185,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,A little eyeshadow video!!! Turn up the volume...,8,1650367221,185,"['hudabeauty', 'sephorasg', 'theprpeople']",3451.0,"[theprpeople, sephorasg, hudabeauty]",[]
186,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,I’ve got a competitor in my house 😩😩😩 The +1 h...,17,1647602185,362,"['osim_sg', 'isaac.com.sg']",9043.0,"[OSIM_SG, osim_sg, isaac.com.sg]","[OSIMuDiva3, OSIMSG, 2+1, Sofa, SmartSofa]"
187,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,TALK SOME MONEY 💰💸 we’re answering some money ...,2,1645708980,169,"['maybanksg', 'isaac.com.sg']",7459.0,"[maybanksg, isaac.com.sg, maybanksgs]","[sp, MaybankSG, MaybankGreenLoans, ForYou]"
188,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,<FENTY BEAUTY GIVEAWAY💓> not my usual style bu...,199,1645452074,291,['fentybeauty'],4887.0,[fentybeauty],[]
189,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,"HAPPY XIN NIAN 😛🧧 as requested, here’s more GR...",3,1643699712,311,"['maybelline', 'charlottetilbury', 'lauramerci...",6662.0,"[maybelline, heroinemakesg, charlottetilbury, ...",[maybellinesg]
190,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,"YOUTUBE: HOW WELL DO WE KNOW EACH OTHER, SPICY...",4,1641823000,390,"['isaac.com.sg', 'yomo_sg']",9433.0,"[yomo_sg, isaac.com.sg]",[]
191,aglimpseofrach,GraphVideo,https://instagram.fsin14-1.fna.fbcdn.net/v/t51...,True,"GRWM WINTER EDITION❄️ NO FOUNDATION, super hyd...",7,1641653169,178,"['urbandecaycosmetics', 'lauramercier', 'laroc...",5954.0,"[firstaidbeauty, heroinemakesg, dermalab_sg, c...",[]
192,aglimpseofrach,GraphVideo,https://instagram.fsin14-2.fna.fbcdn.net/v/t51...,True,"❄️unwind with me, winter edition❄️\nThese prod...",1,1641118333,262,"['lorealparis', 'larocheposay', 'drunkelephant...",7339.0,"[drunkelephant, larocheposay, freshbeauty, lan...",[]


In [28]:
from collections import Counter

def get_influencer_statistics(df):
    """_summary_

    Args:
        df (pd.DataFrame): Profile dataframe

    Returns:
        Dict: Useful information to display
            avg_comments: float
            avg_likes: float
            post_type: dict[type: count] GraphVideo GraphSidecar GraphImage
            avg_video_views: float
            mentions: top 10 mentions
            hashtags: top 10 hashtags
            
    """
    result_dict = {}
    result_dict['avg_comments'] = df['edge_media_to_comment'].mean()
    result_dict['avg_likes'] = df['edge_liked_by'].mean()
    result_dict['post_type'] = dict(df['post_type'].value_counts())
    result_dict['avg_video_views'] = df['video_view_count'].mean()
    all_mentions = []
    for row in df['mentions']:
        # for i in eval(row):
        for i in row:
            all_mentions.append(i)
    mention_counts = sorted(Counter(all_mentions).items(), key=lambda item: -item[1])
    # print(mention_counts)  # debug
    top_mentions = [i[0] for i in mention_counts][:10]
    result_dict['mentions'] = top_mentions
    
    all_hashtags = []
    for row in df['hashtags']:
        # for i in eval(row):
        for i in row:
            all_hashtags.append(i)
    hashtag_counts = sorted(Counter(all_hashtags).items(), key=lambda item: -item[1])
    # print(hashtag_counts)  # debug
    top_hashtags = [i[0] for i in hashtag_counts][:10]
    result_dict['hashtags'] = top_hashtags
    
    mentions_category = [company_cat.get(i) for i in all_mentions if company_cat.get(i)]
    category_counts = Counter(mentions_category)
    # print(category_counts)
    result_dict['category_counts'] = category_counts
    
    all_mentions_cats = [(i, company_cat.get(i))for i in all_mentions if company_cat.get(i)]
    # print(all_mentions_cats)
    if all_mentions_cats:
        user, cat = zip(*all_mentions_cats)
        result_dict['username_cat_df'] = pd.DataFrame({
            'username': user,
            'category': cat 
        })
    else:
        result_dict['username_cat_df'] = pd.DataFrame({
            'username': [],
            'category': [] 
        })
    
    return result_dict
    
influencer_stats = get_influencer_statistics(current_influencer_posts)
influencer_stats

{'avg_comments': 14.36,
 'avg_likes': 305.68,
 'post_type': {'GraphVideo': 22, 'GraphSidecar': 3},
 'avg_video_views': 5850.0,
 'mentions': ['isaac.com.sg',
  'heroinemakesg',
  'benefitcosmeticssg',
  'nudestix',
  'lauramercier',
  'trendyol',
  'charlottetilbury',
  'clarinssg',
  'fentybeauty',
  'maccosmeticssg'],
 'hashtags': ['sp',
  'POV',
  'LazadaSG',
  'OSIMuDiva3',
  'OSIMSG',
  '2+1',
  'Sofa',
  'SmartSofa',
  'MaybankSG',
  'MaybankGreenLoans'],
 'category_counts': Counter({'Beauty Supply Store': 2,
          'Beauty, cosmetic & personal care': 7,
          'Public Relations Agency': 2,
          'Health/beauty': 11,
          'Digital creator': 7,
          'Product/service': 1,
          'Mattress Manufacturer': 1,
          'Furniture': 1,
          'Personal blog': 1,
          'Shopping & retail': 1,
          'Clothing (Brand)': 3,
          'Blogger': 1}),
 'username_cat_df':               username                          category
 0            clarinssg         

In [29]:
# influencer_stats = get_influencer_statistics(current_influencer_posts)
sunburst_fig = px.sunburst(influencer_stats['username_cat_df'], path=['category', 'username'])
sunburst_fig.show()

In [9]:
all_categories = set(company_cat.values())
all_categories

{'Accessories',
 'Actor',
 'Advertising/Marketing',
 'Airport',
 'App page',
 'Apparel & clothing',
 'Appliances',
 'Art',
 'Artist',
 'Arts & entertainment',
 'Athlete',
 'Bakery',
 'Bar',
 'Beauty Salon',
 'Beauty Supply Store',
 'Beauty, cosmetic & personal care',
 'Bed and Breakfast',
 'Biotechnology Company',
 'Blogger',
 'Boutique Store',
 'Brand',
 'Broadcasting & media production company',
 'Building Materials',
 'Business service',
 'Cafe',
 'Camera Operator',
 'Cars',
 'Chinese Restaurant',
 'Clothing (Brand)',
 'Clothing store',
 'Coach',
 'Coffee shop',
 'College & university',
 'Comedian',
 'Community',
 'Community Organization',
 'Cosmetics store',
 'Cosmetology School',
 'Cruise Line',
 'Cycling Studio',
 'DJ',
 'Dance & Night Club',
 'Design & fashion',
 'Designer',
 'Dessert Shop',
 'Digital creator',
 'Dim Sum Restaurant',
 'Education',
 'Electronics',
 'Entrepreneur',
 'Fashion Designer',
 'Fashion Stylist',
 'Fast food restaurant',
 'Fictional Character',
 'Film Dir

In [30]:
from tqdm import tqdm
influencer_stats_list = []
influencer_unique_usernames = influencer_posts_df['username'].unique()

for influencer in tqdm(influencer_unique_usernames):
    current_influencer = influencers_df[influencers_df['username'] == influencer]
    if len(current_influencer) == 1:
        num_followers = current_influencer['num_followers'].iloc[0]
    else:
        continue
        # num_followers = 0
    
    
    current_influencer_posts = influencer_posts_df[influencer_posts_df['username'] == influencer]
    current_influencer_posts = current_influencer_posts.apply(parse_caption, axis=1)
    num_posts = len(current_influencer_posts)
    influencer_stats = get_influencer_statistics(current_influencer_posts)
    to_add = {
        'username': influencer,
        'num_followers': num_followers,
        'avg_comments': influencer_stats['avg_comments'],
        'avg_likes': influencer_stats['avg_likes'],
        'avg_video_views': influencer_stats['avg_video_views'],
        'GraphSidecar': influencer_stats['post_type'].get('GraphSidecar', 0)/num_posts,
        'GraphVideo': influencer_stats['post_type'].get('GraphVideo', 0)/num_posts,
        'GraphImage': influencer_stats['post_type'].get('GraphImage', 0)/num_posts,
    }
    
    cat_dict = {i:influencer_stats['category_counts'].get(i, 0)/num_posts for i in all_categories}
    to_add.update(cat_dict)
    # print(to_add)
    influencer_stats_list.append(to_add)


100%|██████████| 120/120 [00:02<00:00, 42.31it/s]


In [31]:
influencer_stats_df = pd.DataFrame(influencer_stats_list)
influencer_stats_df

,username,num_followers,avg_comments,avg_likes,avg_video_views,GraphSidecar,GraphVideo,GraphImage,Information Technology Company,Household supplies,...,Italian Restaurant,Not a business,Comedian,Outdoor & Sporting Goods Company,Product/service,Fashion Stylist,Photography Videography,Beauty Supply Store,Government organization,Singer
0,huinileee,100647.0,26.846154,4962.846154,7233.461538,0.769231,0.230769,0.000000,0.0,0.0,...,0.0,0.0000,0.000000,0.0,0.307692,0.0,0.000000,0.0,0.0,0.0
1,chankimberly,86223.0,21.785714,2073.214286,8780.928571,0.357143,0.500000,0.142857,0.0,0.0,...,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.071429,0.0,0.0,0.0
2,nxtasha.ann,126428.0,29.380952,2574.809524,14509.904762,0.380952,0.619048,0.000000,0.0,0.0,...,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0
3,jamiepang,157255.0,54.125000,3364.000000,25234.916667,0.333333,0.666667,0.000000,0.0,0.0,...,0.0,0.0000,0.000000,0.0,0.083333,0.0,0.166667,0.0,0.0,0.0
4,buffbaby88,13647.0,15.142857,724.095238,6261.000000,0.238095,0.714286,0.047619,0.0,0.0,...,0.0,0.0000,0.047619,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,pale.nerves,10016.0,7.250000,1034.500000,3901.937500,0.625000,0.375000,0.000000,0.0,0.0,...,0.0,0.0625,0.000000,0.0,0.125000,0.0,0.000000,0.0,0.0,0.0
108,kylolizy,51191.0,10.208333,1087.416667,10122.958333,0.083333,0.833333,0.083333,0.0,0.0,...,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
109,hargaohunk,16153.0,17.750000,752.250000,7206.875000,0.208333,0.750000,0.041667,0.0,0.0,...,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
110,chiaraang,13832.0,87.791667,749.375000,7831.000000,0.333333,0.666667,0.000000,0.0,0.0,...,0.0,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0


In [14]:
# influencer_stats_df.to_csv('data/influencer_stats.csv', index=False)